In [1]:
from fastai.imports import *
from fastai.vision import *
import tifffile as tiff
from skimage.external import tifffile as sktif
from joblib import Parallel, delayed
import torch.functional as F

import functools, traceback
def gpu_mem_restore(func):
    "Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

os.environ['FASTAI_TB_CLEAR_FRAMES']="1"

In [2]:
data_dir = Path("/media/wwymak/Storage/urban-3D-satellite")
train_dir = data_dir / "training"
cropped_dir =  data_dir / "cropped_training"
cropped_val_dir =  data_dir / "cropped_validation"

In [10]:
def get_img_id(fname):
    fname = fname.replace('_RGB.tif', '')
    img_id = "_".join(fname.split('_')[:-1])
    return img_id

train_img_ids = [get_img_id(f.name) for f in cropped_dir.ls() if f.name.endswith('RGB.tif')]
val_img_ids = [get_img_id(f.name) for f in cropped_val_dir.ls() if f.name.endswith('RGB.tif')]

In [25]:
labels_train_dir = data_dir / "cropped_training_labels"
labels_train_dir.mkdir(exist_ok=True)

labels_val_dir = data_dir / "cropped_validation_labels"
labels_val_dir.mkdir(exist_ok=True)

In [70]:
def parse_label(arr, label_val):
    item = np.argwhere(arr==label_val)
    x1, x2, y1, y2 = item[:,0].min(), item[:,0].max(), item[:,1].min(), item[:,1].max()
    x = int(0.5 * (x1+ x2))
    y = int(0.5 * (y1+y2) )
    w = x2 -x1
    h = y2-y1
    return x, y, w, h
def label_to_str(label_out_tuple):
    return " ".join(list(map(str, label_out_tuple)))

def parse_img(img_fname, dataset_type="training"):
    label_img = sktif.imread(str(data_dir / f"cropped_{dataset_type}"/ img_fname))
    label_filepath =data_dir / f"cropped_{dataset_type}_labels/{img_fname.replace('.tif', '.txt')}"
    max_label = label_img.max()
    
    min_label = np.min(label_img[np.nonzero(label_img)]) if max_label > 0 else 0
    valid_labels = [x for x in range(min_label, max_label + 1) if len(np.where(label_img == x)[0]) > 0]
    
    with open(label_filepath, 'w') as f:
        [f.write(f"{img_fname.replace('.tif','')} {label_to_str(parse_label(label_img, label_val))}\n") \
         for label_val in valid_labels]
    
    
    

In [48]:
cropped_train_fnamescropped_train_fnames =[f.name for f in ( data_dir / "cropped_training").ls() if f.name.endswith('GTI.tif')]
cropped_val_fnames =[f.name for f in ( data_dir / "cropped_validation").ls() if f.name.endswith('GTI.tif')]

In [71]:
%time out=[parse_img(fname) for fname in cropped_train_fnames]

CPU times: user 18.1 s, sys: 114 ms, total: 18.2 s
Wall time: 18.2 s


In [72]:
%time out=[parse_img(fname,  dataset_type="validation") for fname in cropped_val_fnames]

CPU times: user 3.77 s, sys: 35.4 ms, total: 3.8 s
Wall time: 3.8 s
